<h1>
    <br style ="color:#EA7600"> UCL </br> 
    </p>
</h1>

<div>

  <h4>
      <p style="font-size:18pt">MSIN0166 Data Engineering</p>
      <p style="font-size:18pt">Introduction to APIs</p>
  </h4>

</div>

In [1]:
!pip install python-dotenv;


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.



### Saving an Environment Variable

In [2]:
import requests
import os
import json
from dotenv import load_dotenv

load_dotenv() # take environment variable from .env

# To set your enviornment variables in your terminal run the following line:
bearer_token = os.getenv("API_KEY")

In [3]:
# Source: Twitter dev github
# https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Filtered-Stream/filtered_stream.py

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )

### Update Rules

In [4]:
get_rules();

In [5]:
delete_all_rules(get_rules());

In [6]:
get_rules()

{'meta': {'sent': '2022-03-22T19:47:02.327Z', 'result_count': 0}}

In [7]:
import requests
import json

# Lets define a function to pass the headers with our bearer_token
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [8]:
import requests
import json

# Let's define another function to set the rules to filter the live stream of tweets
def set_rules(headers, sample_rules):
    # You can adjust the rules if needed
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    #print(json.dumps(response.json()))

In [9]:
import requests
import json

# Let's set the code to get the stream of tweets
def get_stream(headers, max_number_tweets):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,
    )
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    counter_tweets = 0
    stream_tweets = {'stream_tweets':[]}

    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            stream_tweets["stream_tweets"].append(json_response)

            #print(json.dumps(json_response, indent=4, sort_keys=True))
            counter_tweets += 1

            if counter_tweets >= max_number_tweets:
                with open("atp_tweets_final.json", 'a') as f:
                    json.dump(stream_tweets, f, indent=4)

                # Close the stream of tweets
                response.close()
                
                # Break the loop
                break

### Scraping using Twitter API

In [10]:
import time
start = time.time()
#print("hello")

# Create the headers for the HTTP request
headers = create_headers(bearer_token)

# Set the rules for the filtering of tweets that mention the Indian Wells tournament
# Filtering the time of the tournament and the top 5 players from singles and double each

sample_rules = [
    {"value": "BNPPARIBASOPEN lang:en -is:retweet", "activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"}, # tournament name
    {"value": "IndianWells lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "BNPP022 lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Daniil Medvedev lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"}, # Top 5 single player name
    {"value": "Novak Djokovic lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Alexander Zverev lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Rafael Nadal lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Stefanos Tsitsipas lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Mate Pavic lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"}, # Top 5 double player name
    {"value": "Nikola Mektic lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Joe Salisbury lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Rajeev Ram lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
    {"value": "Horacio Zeballos lang:en -is:retweet","activity_start_time": "2022-03-07T00:00:00Z", "activity_end_time": "2022-03-20T23:59:59Z"},
]

#@BNPPARIBASOPEN #IndianWells #BNPPO22

set_rules_add = set_rules(headers, sample_rules)

# Get the stream of tweets live
max_number_tweets = 10
get_stream(headers, max_number_tweets)


### Save it into a JSON file

In [11]:
import json 

with open('atp_tweets_final.json') as json_file:
    data = json.load(json_file)

#print(data)